# Cleaning oferta empleo

<a id = 'table'> </a>
## Table of contents

- Part I - Gathering the data
    - [Key Points](#key)
    - [Converting DTA to CSV file](#dta)
    - [Reading the CSV file](#read)    

- Part II - Assessing and cleaning
    - [Exploring the table](#explore)
    - [Making copies](#copies)
    - [Assesment and cleaning](#clean)
    - [Storing Data](#store)
    
- Part III - Analysis and visualization
    - [Gráfica 1](#g1)
    - [Gráfica 2](#g2)
    - [Gráfica 3](#g3)

In [2]:
import pandas as pd
import numpy as np
import os
import time
import sys
import re

# visualizations
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline

# for visualizing all columns
pd.options.display.max_columns = None

<a id='key'></a>
[Return to Table of Contents](#table)

## Key Points for assessing and cleaning

- The BID2.OFERTA_EMPLEO.dta file contains almost  --- records and --- fields (columns).
- The analysis will be delimited to the next fields: 
    - 0-idtabla
    - 1-id_oferta_empleo
    - 2-id_empresa
    - 5-c_arne_id
    - 6-c_arne_des
    - 8-id_ocupacion
    - 10-funciones
    - 11-dias_laborales
    - 12-hora_entrada
    - 13-hora_salida
    - 14-rolar_turno
    - 16-salario
    - 17-id_tipo_contrato
    - 18-tipcont_opcion
    - 19-numero_plazas
    - 20-id_causa_vacante
    - 21-causa_origen_vacante
    - 22-disponibilidad_viajar
    - 23-disponibilidad_radicar
    - 24-id_nivel_estudio
    - 25-nivel_estudio
    - 48-experiencia
    - 49-des_conocimiento
    - 50-edad_minima
    - 51-edad_maxima
    - 52-genero
    - 53-desc_genero
    - 58-codigo_postal
    - 59-id_entidad_dom
    - 60-id_municipio

<a id='dta'></a>
[Return to Table of Contents](#table)
## Converting DTA to CSV file

In [62]:
#  First we create a reader with a small chunk in order to know the column names
my_stata_reader = pd.read_stata('data/BID2.OFERTA_EMPLEO.dta', chunksize=2)
my_stata_reader._encoding = 'utf-8'
my_stata_reader.variable_labels()

{'idtabla': 'IdTabla',
 'id_oferta_empleo': 'ID_OFERTA_EMPLEO',
 'id_empresa': 'ID_EMPRESA',
 'fecha_alta': 'FECHA_ALTA',
 'ext_fecha_modificacion': 'EXT_FECHA_MODIFICACION',
 'c_arne_id': 'c_ArNe_id',
 'c_arne_des': 'c_ArNe_Des',
 'titulo_oferta': 'TITULO_OFERTA',
 'id_ocupacion': 'ID_OCUPACION',
 'ext_ocupacion': 'EXT_OCUPACION',
 'funciones': 'FUNCIONES',
 'dias_laborales': 'DIAS_LABORALES',
 'hora_entrada': 'HORA_ENTRADA',
 'hora_salida': 'HORA_SALIDA',
 'rolar_turno': 'ROLAR_TURNO',
 'empresa_ofrece': 'EMPRESA_OFRECE',
 'salario': 'SALARIO',
 'id_tipo_contrato': 'ID_TIPO_CONTRATO',
 'tipcont_opcion': 'TIPCONT_OPCION',
 'numero_plazas': 'NUMERO_PLAZAS',
 'id_causa_vacante': 'ID_CAUSA_VACANTE',
 'causa_origen_vacante': 'CAUSA_ORIGEN_VACANTE',
 'disponibilidad_viajar': 'DISPONIBILIDAD_VIAJAR',
 'disponibilidad_radicar': 'DISPONIBILIDAD_RADICAR',
 'id_nivel_estudio': 'ID_NIVEL_ESTUDIO',
 'nivel_estudio': 'NIVEL_ESTUDIO',
 'id_situacion_academica': 'ID_SITUACION_ACADEMICA',
 'desc_situ

In [20]:
# The my_stata_reader object allows us reading the big dta file by chunks
# filtering by the columns that we need 
my_stata_reader = pd.read_stata('data/BID2.OFERTA_EMPLEO.dta', chunksize=10000, 
                                columns=['idtabla','id_oferta_empleo','id_empresa','fecha_alta','ext_fecha_modificacion','c_arne_des',
                                         'id_ocupacion','ext_ocupacion','funciones','dias_laborales','hora_entrada','hora_salida',
                                         'rolar_turno','salario','tipcont_opcion',
                                         'numero_plazas','causa_origen_vacante',
                                         'disponibilidad_viajar','disponibilidad_radicar',
                                         'nivel_estudio','experiencia','des_conocimiento','edad_minima',
                                         'edad_maxima','desc_genero'])
my_stata_reader._encoding = 'utf-8'

In [21]:
# start and end will measure time
start = time.time()
first = True

# for each chunk it will do the next:
for chunk in my_stata_reader:
    if first:        # only append the header if it is the first chunk:
        z = pd.DataFrame(chunk)
        z.to_csv('oferta_empleo.csv', mode='w', index=False, header=True)  #‘w’ write new file
        print('.', end = '')
        first = False
    else:    # Omit the header for the next chunks
        z = pd.DataFrame(chunk)
        z.to_csv('oferta_empleo.csv', mode='a', index=False, header=False) #‘a’ append
    
    # Print a dot each time a chunk is appended
    print('.', end = '')
end = time.time()
print (('Total time: ') + str(end - start))

...............................................................................................................................Total time: 76.05942678451538


In [22]:
# Verifiquemos el tamanio del CSV
filesize= os.stat('oferta_empleo.csv').st_size
print(filesize)

461083422


<a id='read'></a>
[Return to Table of Contents](#table)
## Reading the CSV

The successful use of pd.read_csv may vary because of the <b>file size, type of processor, memory, code,</b> or some other variables.

If the code runs out of memory, you can try reading in chunks.

In [23]:
oferta = pd.read_csv('oferta_empleo.csv' )
oferta.head(1)

,idtabla,id_oferta_empleo,id_empresa,fecha_alta,ext_fecha_modificacion,c_arne_des,id_ocupacion,ext_ocupacion,funciones,dias_laborales,hora_entrada,hora_salida,rolar_turno,salario,tipcont_opcion,numero_plazas,causa_origen_vacante,disponibilidad_viajar,disponibilidad_radicar,nivel_estudio,experiencia,des_conocimiento,edad_minima,edad_maxima,desc_genero
0,1,1.0,23745601.0,2017-01-09,2017-01-09,Apoyo de Oficina,311104,Secretaria recepcionista,NaN,0,9,18,1,80.0,Ninguno,1,Sin registrar,1,1,SECUNDARIA/SEC. TÉCNICA,0,NINGUNO,35,35,NO APLICA


<a id='explore'></a>
[Return to Table of Contents](#table)
## Exploring the table


In [7]:
oferta.sample(3)

,idtabla,id_oferta_empleo,id_empresa,c_arne_des,id_ocupacion,ext_ocupacion,funciones,dias_laborales,hora_entrada,hora_salida,rolar_turno,salario,tipcont_opcion,numero_plazas,causa_origen_vacante,disponibilidad_viajar,disponibilidad_radicar,nivel_estudio,experiencia,des_conocimiento,edad_minima,edad_maxima,desc_genero
189424,189425,2576518.0,1.529463e+09,Oficios y Servicios,511101,Cocinera,PREPARACIÓN DE TODO TIPO DE ALIMENTOS FRÍA CAL...,1101111,11,20,2,4000.0,Contrato por tiempo indeterminado,5,Empresa nueva,1,1,PROFESIONAL TÉCNICO (CONALEP),2,NINGUNO,18,45,NO APLICA
1047769,1047770,3180146.0,1.511719e+09,Ventas,423101,Promotor de ventas,ATENCION A CLIENTES CAMBACEO OFERTA DE PROMOCI...,111111,3,20,2,2403.0,Contrato por tiempo indeterminado,5,Puesto de nueva creación,1,1,SECUNDARIA/SEC. TÉCNICA,1,NINGUNO,18,55,NO APLICA
554455,554456,2717375.0,1.513336e+09,Oficios y Servicios,713307,Instalador de cristales,APOYO EN LA INSTALACIÓN DE CRISTAL AUTOMOTRIZ ...,111111,9,18,2,3800.0,Contrato por tiempo indeterminado,4,Reposición de personal,1,1,SECUNDARIA/SEC. TÉCNICA,1,NINGUNO,18,30,NO APLICA


oferta.groupby("dias_laborales")["id_oferta_empleo"].count()

In [8]:
oferta.shape

(1250397, 23)

In [27]:
oferta.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1250397 entries, 0 to 1250396
Data columns (total 25 columns):
idtabla                   1250397 non-null int64
id_oferta_empleo          1250397 non-null float64
id_empresa                1250397 non-null float64
fecha_alta                1250397 non-null object
ext_fecha_modificacion    1250397 non-null object
c_arne_des                1250397 non-null object
id_ocupacion              1250397 non-null int64
ext_ocupacion             1250397 non-null object
funciones                 1219658 non-null object
dias_laborales            1250397 non-null int64
hora_entrada              1250397 non-null int64
hora_salida               1250397 non-null int64
rolar_turno               1250397 non-null int64
salario                   1250397 non-null float64
tipcont_opcion            1250397 non-null object
numero_plazas             1250397 non-null int64
causa_origen_vacante      1250397 non-null object
disponibilidad_viajar     1250397 non-nul

In [56]:
oferta.groupby('id_oferta_empleo')['idtabla'].count()

id_oferta_empleo
1.0          2
2.0          2
3.0          2
4.0          2
5.0          2
6.0          2
7.0          2
8.0          2
9.0          2
10.0         2
11.0         2
12.0         2
13.0         2
14.0         2
15.0         2
16.0         2
17.0         2
18.0         2
19.0         2
20.0         2
21.0         2
22.0         2
23.0         2
24.0         2
25.0         2
26.0         2
27.0         2
28.0         2
29.0         2
30.0         2
            ..
3364322.0    1
3364323.0    1
3364324.0    1
3364325.0    1
3364326.0    1
3364327.0    1
3364328.0    1
3364329.0    1
3364330.0    1
3364331.0    1
3364332.0    1
3364333.0    1
3364334.0    1
3364335.0    1
3364336.0    1
3364337.0    1
3364338.0    1
3364339.0    1
3364340.0    1
3364341.0    1
3364342.0    1
3364343.0    1
3364344.0    1
3364345.0    1
3364346.0    1
3364347.0    1
3364348.0    1
3364349.0    1
3612541.0    1
3613910.0    1
Name: idtabla, Length: 1217159, dtype: int64

In [57]:
oferta.sort_values(by='id_oferta_empleo')

,idtabla,id_oferta_empleo,id_empresa,fecha_alta,ext_fecha_modificacion,c_arne_des,id_ocupacion,ext_ocupacion,funciones,dias_laborales,hora_entrada,hora_salida,rolar_turno,salario,tipcont_opcion,numero_plazas,causa_origen_vacante,disponibilidad_viajar,disponibilidad_radicar,nivel_estudio,experiencia,des_conocimiento,edad_minima,edad_maxima,desc_genero
0,1,1.0,2.374560e+07,2017-01-09,2017-01-09,Apoyo de Oficina,311104,Secretaria recepcionista,NaN,0,9,18,1,80.0,Ninguno,1,Sin registrar,1,1,SECUNDARIA/SEC. TÉCNICA,0,NINGUNO,35,35,NO APLICA
235628,235629,1.0,1.653120e+07,2016-01-08,2016-01-08,Apoyo de Oficina,311104,Secretaria recepcionista,"ARCHIVO, FACTURACION Y ATENCION A CLIENTES",1111110,9,17,1,3000.0,Contrato por tiempo determinado,1,Puesto de nueva creación,1,1,PREPA O VOCACIONAL,1,X,35,35,HOMBRE
235667,235668,2.0,1.172201e+06,2016-01-14,2016-01-14,Oficios y Servicios,973202,SIN ESPECIFICAR,Sin Registro de funciones,0,9,18,1,73.0,Ninguno,1,Sin registrar,1,1,SECUNDARIA/SEC. TÉCNICA,0,Sin registro,35,35,NO APLICA
235666,235667,2.0,2.542601e+06,2017-01-09,2017-01-09,Sector Salud / Medicina,973202,SIN ESPECIFICAR,NaN,0,9,18,1,80.0,Ninguno,1,Sin registrar,1,1,SECUNDARIA/SEC. TÉCNICA,0,NINGUNO,35,35,NO APLICA
7,8,3.0,4.054010e+05,2016-01-14,2016-01-14,"Instalación, Mantenimiento, y Reparación",821103,Ensamblador de partes y motores automotrices,"MANEJO DE MAQUINADO, ENSAMBLE DE POLEAS Y ALUM...",1111110,8,16,1,4308.0,Contrato por tiempo indeterminado,65,Empresa nueva,1,1,SECUNDARIA/SEC. TÉCNICA,0,NINGUNO,18,33,NO APLICA
1,2,3.0,4.054010e+05,2017-01-09,2017-01-09,Servicios bancarios y financieros,821103,Ensamblador de partes y motores automotrices,NaN,0,9,18,1,80.0,Ninguno,1,Sin registrar,1,1,SECUNDARIA/SEC. TÉCNICA,0,NINGUNO,35,35,NO APLICA
235629,235630,4.0,2.987301e+06,2017-01-09,2017-01-09,Sector Salud / Medicina,821103,Ensamblador de partes y motores automotrices,NaN,1111100,9,15,1,5000.0,Contrato por tiempo indeterminado,2,Puesto de nueva creación,1,1,SABER LEER Y ESCRIBIR,0,NINGUNO,16,55,NO APLICA
235662,235663,4.0,1.653130e+07,2016-01-14,2016-01-14,"Instalación, Mantenimiento, y Reparación",821103,Ensamblador de partes y motores automotrices,ENSAMBLE GENERAL DE PARTES DE AUTOMOVILES,1111110,7,17,1,6867.0,Contrato por tiempo indeterminado,30,Otra,1,1,SECUNDARIA/SEC. TÉCNICA,0,NINGUNO,18,40,HOMBRE
4,5,5.0,7.712010e+05,2017-01-09,2017-01-09,Sector Salud / Medicina,731102,Artesano de madera,NaN,1111100,9,15,1,5000.0,Contrato por tiempo indeterminado,3,Necesidades temporales de mano de obra,1,1,SABER LEER Y ESCRIBIR,0,NINGUNO,16,55,NO APLICA
235663,235664,5.0,1.272800e+07,2016-01-14,2016-01-14,Ventas,731102,Artesano de madera,"ELABORACION DE ALAJEROS, SERVILLETEROS, MESAS ...",1111100,14,20,1,2000.0,Contrato por tiempo indeterminado,25,Empresa nueva,1,1,SABER LEER Y ESCRIBIR,0,CARPINTERIA,35,35,NO APLICA


In [ ]:
oferta.id_empresa.unique()

In [ ]:
oferta.disponibilidad_radicar.unique()

In [ ]:
oferta.c_arne_des.unique()

In [ ]:
oferta.id_ocupacion.unique()

In [ ]:
oferta.ext_ocupacion.unique()

In [ ]:
oferta.funciones.unique().tolist()

In [ ]:
oferta.dias_laborales.unique()

In [50]:
a = oferta.hora_entrada.unique()
print(sorted(a))

[2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24]


In [51]:
a = oferta.hora_salida.unique()
print(sorted(a))

[2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24]


In [ ]:
oferta.rolar_turno.unique()

In [ ]:
oferta.salario.unique()

In [ ]:
oferta.tipcont_opcion.unique()

In [ ]:
oferta.numero_plazas.unique()

In [ ]:
oferta.causa_origen_vacante.unique()

In [ ]:
oferta.disponibilidad_viajar.unique()

In [ ]:
oferta.disponibilidad_radicar.unique()

In [ ]:
oferta.nivel_estudio.unique().tolist()

In [ ]:
oferta.experiencia.unique()

In [ ]:
oferta.des_conocimiento.unique().tolist()

In [ ]:
oferta.edad_minima.unique()

In [ ]:
oferta.edad_maxima.unique()

In [27]:
oferta.desc_genero.unique()

array(['NO APLICA', 'HOMBRE', 'MUJER'], dtype=object)

In [ ]:
#NaN in candidatos
candidatos = candidatos.replace("nan",np.nan)
msno.matrix(candidatos.sample(250));

<a id='copies'></a>
[Return to Table of Contents](#table)
## Making copies

The first step before any cleaning is **always make copies**

In [28]:
oferta_clean = oferta.copy()

<a id='clean'></a>[Return to Table of Contents](#table)
## Assesment and cleaning
#### Quality changes
1. [Wrong Datatypes](#datatypes)
2. [Fill blank spaces with zeros to the left in `dias_laborales`](#zeros)
3. [3 ](#edomex)
4. [4](#edad)
5. [5](#trimAndUpp)
6. [6](#unify)
7. [7](#lengua_indigena)
8. [8](#uplowcase)  
9. [9](#col_names)
10. [10](#categorical)

#### Tidiness changes
Not a single so far.

<a id='datatypes'></a>
### 1. Wrong datatypes 

- `id_oferta_empleo` needs a data type change from float to int
- `id_empresa` needs a data type change from float to int
- `dias_laborales` needs a data type change from int to string

In [58]:
oferta_clean['id_oferta_empleo'] =  oferta['id_oferta_empleo'].astype(int)
oferta_clean['id_empresa'] =  oferta['id_empresa'].astype(int)
oferta_clean['dias_laborales'] =  oferta['dias_laborales'].astype(str)

In [33]:
oferta_clean.id_oferta_empleo.dtype

dtype('int64')

In [34]:
oferta_clean.id_empresa.dtype

dtype('int64')

In [35]:
oferta_clean.dias_laborales.dtype

dtype('O')

<a id='zeros'></a>
[Return to Assesment](#assess1)
### 2. Fill blank spaces with zeros to the left in `dias_laborales` 
Using str.zfill function

In [36]:
oferta_clean.dias_laborales = oferta_clean.dias_laborales.str.zfill(7)

In [37]:
oferta_clean.dias_laborales.unique()

array(['0000000', '1111100', '1111110', '0111100', '0111111', '1011111',
       '1101111', '0111110', '1111011', '1111111', '0111011', '1110111',
       '1111101', '0010000', '0101010', '1000011', '0100000', '1011011',
       '0101111', '1010101', '0110111', '0000001', '0100001', '1000111',
       '1100111', '0011111', '1111000', '1100000', '1000001', '1100001',
       '0110100', '1110110', '1100011', '0110110', '1101011', '1011101',
       '1000000', '0111101', '0110000', '1010010', '0010101', '1101010',
       '0000011', '1001111', '0010100', '1110101', '1101101', '0100010',
       '1010001', '1100100', '1001010', '0111000', '0101011', '1111001',
       '0111010', '0000111', '0111001', '0011110', '1011001', '1010000',
       '0001111', '1001011', '0100100', '0101110', '0101001', '1101100',
       '0001000', '1110011', '1010011', '1111010', '0101000', '1110001',
       '1011110', '0000010', '1001001', '0010110', '0011000', '1001110',
       '0011100', '0010010', '0001101', '0110011', 

<a id='edomex'></a>
[Return to Assesment](#assess1)
### 3. The value 'MEXICO, ESTADO DE'  must be changed 
'ESTADO DE MEXICO' must be used instead of 'MEXICO, ESTADO DE', which is a better reading of the value.
Use replace function

In [61]:
oferta_clean.sort_values(by='id_oferta_empleo')

,idtabla,id_oferta_empleo,id_empresa,fecha_alta,ext_fecha_modificacion,c_arne_des,id_ocupacion,ext_ocupacion,funciones,dias_laborales,hora_entrada,hora_salida,rolar_turno,salario,tipcont_opcion,numero_plazas,causa_origen_vacante,disponibilidad_viajar,disponibilidad_radicar,nivel_estudio,experiencia,des_conocimiento,edad_minima,edad_maxima,desc_genero
0,1,1,23745601,2017-01-09,2017-01-09,Apoyo de Oficina,311104,Secretaria recepcionista,NaN,0,9,18,1,80.0,Ninguno,1,Sin registrar,1,1,SECUNDARIA/SEC. TÉCNICA,0,NINGUNO,35,35,NO APLICA
235628,235629,1,16531201,2016-01-08,2016-01-08,Apoyo de Oficina,311104,Secretaria recepcionista,"ARCHIVO, FACTURACION Y ATENCION A CLIENTES",1111110,9,17,1,3000.0,Contrato por tiempo determinado,1,Puesto de nueva creación,1,1,PREPA O VOCACIONAL,1,X,35,35,HOMBRE
235667,235668,2,1172201,2016-01-14,2016-01-14,Oficios y Servicios,973202,SIN ESPECIFICAR,Sin Registro de funciones,0,9,18,1,73.0,Ninguno,1,Sin registrar,1,1,SECUNDARIA/SEC. TÉCNICA,0,Sin registro,35,35,NO APLICA
235666,235667,2,2542601,2017-01-09,2017-01-09,Sector Salud / Medicina,973202,SIN ESPECIFICAR,NaN,0,9,18,1,80.0,Ninguno,1,Sin registrar,1,1,SECUNDARIA/SEC. TÉCNICA,0,NINGUNO,35,35,NO APLICA
7,8,3,405401,2016-01-14,2016-01-14,"Instalación, Mantenimiento, y Reparación",821103,Ensamblador de partes y motores automotrices,"MANEJO DE MAQUINADO, ENSAMBLE DE POLEAS Y ALUM...",1111110,8,16,1,4308.0,Contrato por tiempo indeterminado,65,Empresa nueva,1,1,SECUNDARIA/SEC. TÉCNICA,0,NINGUNO,18,33,NO APLICA
1,2,3,405401,2017-01-09,2017-01-09,Servicios bancarios y financieros,821103,Ensamblador de partes y motores automotrices,NaN,0,9,18,1,80.0,Ninguno,1,Sin registrar,1,1,SECUNDARIA/SEC. TÉCNICA,0,NINGUNO,35,35,NO APLICA
235629,235630,4,2987301,2017-01-09,2017-01-09,Sector Salud / Medicina,821103,Ensamblador de partes y motores automotrices,NaN,1111100,9,15,1,5000.0,Contrato por tiempo indeterminado,2,Puesto de nueva creación,1,1,SABER LEER Y ESCRIBIR,0,NINGUNO,16,55,NO APLICA
235662,235663,4,16531301,2016-01-14,2016-01-14,"Instalación, Mantenimiento, y Reparación",821103,Ensamblador de partes y motores automotrices,ENSAMBLE GENERAL DE PARTES DE AUTOMOVILES,1111110,7,17,1,6867.0,Contrato por tiempo indeterminado,30,Otra,1,1,SECUNDARIA/SEC. TÉCNICA,0,NINGUNO,18,40,HOMBRE
4,5,5,771201,2017-01-09,2017-01-09,Sector Salud / Medicina,731102,Artesano de madera,NaN,1111100,9,15,1,5000.0,Contrato por tiempo indeterminado,3,Necesidades temporales de mano de obra,1,1,SABER LEER Y ESCRIBIR,0,NINGUNO,16,55,NO APLICA
235663,235664,5,12728001,2016-01-14,2016-01-14,Ventas,731102,Artesano de madera,"ELABORACION DE ALAJEROS, SERVILLETEROS, MESAS ...",1111100,14,20,1,2000.0,Contrato por tiempo indeterminado,25,Empresa nueva,1,1,SABER LEER Y ESCRIBIR,0,CARPINTERIA,35,35,NO APLICA


In [60]:
oferta_clean.hora_entrada.unique()

array([ 9, 10,  8, 15,  7, 14, 11,  6, 16, 13, 17, 12, 24, 18, 21, 20,  3,
       23,  5, 19,  4,  2, 22])

In [ ]:
oferta.hora_salida.unique()

In [ ]:
oferta_clean.rolar_turno.unique()

In [ ]:
type(oferta_clean.rolar_turno)

In [ ]:
oferta_clean.rolar_turno=oferta.rolar_turno.astype(int)

In [ ]:
#Verify change
oferta_clean.head(5)


In [ ]:
# Verify change
oferta_clean.hora_entrada.unique()


<a id='edad'></a>
[Return to Assesment](#assess1)
### 4. Some candidates have unexpected values for `edad`.
Values from 3 to 15 years will be changed for the average age.

In [ ]:
# The average age
candidatos_clean.edad.mean()

A numpy array will be created and then replaced where edad < 16

In [ ]:
a = np.array(candidatos_clean.edad.values.tolist())

In [ ]:
candidatos_clean.edad = np.where(a < 16, candidatos_clean.edad.mean(), a)

In [ ]:
# Verify the change
candidatos_clean.query('edad < 16')

<a id='trimAndUpp'></a>
[Return to Assesment](#assess1)
### 5. Values in `tipoexperiencia` have mixed upper-lowercase and spaces

Use the functions `strip` and `upper`

In [ ]:
candidatos_clean.tipoexperiencia = candidatos_clean.tipoexperiencia.str.strip()

In [ ]:
candidatos_clean.tipoexperiencia = candidatos_clean.tipoexperiencia.str.upper()

In [ ]:
# Verify the changes
candidatos_clean.tipoexperiencia.value_counts()

<a id='unify'></a>
[Return to Assesment](#assess1)
### 6. Values for no experience in `tipoexperiencia` can be unified

Use the match function and regular expressions (regex)

In [ ]:
# match all NING
a = candidatos_clean[candidatos_clean['tipoexperiencia'].str.match(pat = r'.*NING.*', case=False)].tipoexperiencia

In [ ]:
# show all and count values
with pd.option_context('display.max_rows', None, 'display.max_columns', None):
    print(a.value_counts())

In [ ]:
# a serie with all values of tipoexperiencia
experiencias = candidatos_clean.tipoexperiencia
type(experiencias)

In [ ]:
# r is a regex NING|ning
r = re.compile('NING|ning|Ning')
# vectorized function which takes a nested sequence of objects or numpy arrays as inputs 
# and returns a single numpy array
# Using re.search instead of re.match which matches the pattern from beginning of the string
ning_match = np.vectorize(lambda x:bool(r.search(x)))
# call function for the series experiencias. Returns a bool array
exp_bool_array = ning_match(experiencias)

In [ ]:
exp_bool_array

In [ ]:
# Setting values with df.loc
# https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.loc.html
# convert the bool array to list. Change all True in tipoexperiencia with 'NINGUNA'
candidatos_clean.loc[exp_bool_array.tolist(),['tipoexperiencia']] = 'NINGUNA'

Also we'll add up 'NAN' and '\[INDICA EL CONOCIMIENTO\]' to 'NO ESPECIFICADO'

In [ ]:
candidatos_clean = candidatos_clean.replace({'tipoexperiencia':{'NAN':'NO ESPECIFICADO', 
                                                                '[INDICA EL CONOCIMIENTO]':'NO ESPECIFICADO'}});

Verifying changes

In [ ]:
# match all
b = candidatos_clean[candidatos_clean['tipoexperiencia'].str.match(pat = r'.*NING.*', case=False)].tipoexperiencia

In [ ]:
# show all and count values
with pd.option_context('display.max_rows', None, 'display.max_columns', None):
    print(b.value_counts())

In [ ]:
candidatos_clean.tipoexperiencia.value_counts()

<a id='lengua_indigena'></a>
[Return to Assesment](#assess1)
### 7. Values in `clen_idcvelengua` must be completed

In [ ]:
candidatos_clean.clen_idcvelengua.unique()

In [ ]:
candidatos_clean = candidatos_clean.replace({'clen_idcvelengua':{'Si, NAHUAT':'Nahuatl','Si, TZELTZ':'Tzeltal',
                                        'Si, TOTONA':'Totonaca', 'Si, TZOTZI':'Tzotzil','Si, POPOLU':'Popoluca',
                                        'Si, TOJOLA':'Tojolabal', 'Si, PUREPE':'Purepecha', 'Si, POPOLO':'Popoloca',
                                        'Si, CHOL':'Chol', 'Si, HUASTE':'Huasteco', 'Si, OTRAS ':'Otras', 
                                        'Si, NO ESP':'No Español', 'Si, MIXTEC':'Mixteco', 'Si, ZAPOTE':'Zapoteco',
                                        'Si, MAZATE':'Mazateco', 'Si, AGUACA':'Aguacateco','Si, OTOMI':'Otomí',
                                        'Si, MAZAHU':'Mazahua', 'Si, DE LAL':'De Lal', 'Si, PAPAGO':'Pápago', 
                                        'Si, SERI':'Seri', 'Si, HUICHO':'Huichol', 'Si, CHINAN':'Chinanteco', 
                                        'Si, TRIQUI':'Triqui', 'Si, AMUZGO':'Amuzgo','Si, MOTOCI':'Motozintleco',
                                        'Si, ZOQUE':'Zoque', 'Si, MAYA':'Maya', 'Si, MIXE':'Mixe', 
                                        'Si, CHATIN':'Chatino', 'Si, TLAPAN':'Tlapaneco', 'Si, CUCAPA':'Cucapa', 
                                        'Si, MATLAT':'Matlatzinca', 'Si, LACAND':'Lacandón', 'Si, MAYO':'Mayo' }})

In [ ]:
candidatos_clean = candidatos_clean.replace({'clen_idcvelengua':{'NO':'No', '0':'No'}})

In [ ]:
candidatos_clean.clen_idcvelengua.unique()

<a id='uplowcase'></a>
[Return to Assesment](#assess1)
### 8. `area` and `subarea` have mixed upper and lowercase values


In [ ]:
candidatos_clean.area = candidatos_clean.area.str.upper()
candidatos_clean.subarea = candidatos_clean.subarea.str.upper()

In [ ]:
# Verify there are no lowercase values in area
unique2, counts2 = np.unique(candidatos_clean.area, return_counts=True)
dict(zip(unique2, counts2))

In [ ]:
# Verify there are no lowercase values in subarea
unique3, counts3 = np.unique(candidatos_clean.subarea, return_counts=True)
dict(zip(unique3, counts3))

<a id='col_names'></a>
[Return to Assesment](#assess1)
### 9. Column names  


It is suggested to use the rename function to change names to the following:

- edocivil -> edo_civil
- qempleobusca -> empleo_buscado
- salariopretente -> salario_pretendido
- tipoexperiencia -> tipo_experiencia
- trabajaactualmente ->trabaja_actualmente
- motivo  -> motivo_no_trabaja 
- fchcomienzo_busqueda -> fecha_inicio_busqueda
- clen_idcvelengua -> lengua_indigena
- tipodiscapacidad -> tipo_discapacidad 

In [ ]:
oferta_clean.rename(columns={'edocivil':'edo_civil',
                                  'qempleobusca':'empleo_buscado',
                                  'salariopretente':'salario_pretendido',
                                  'tipoexperiencia':'tipo_experiencia',
                                  'trabajaactualmente':'trabaja_actualmente',
                                  'motivo':'motivo_no_trabaja',
                                  'fchcomienzo_busqueda':'fecha_inicio_busqueda',
                                  'clen_idcvelengua':'lengua_indigena',
                                  'tipodiscapacidad':'tipo_discapacidad'}, inplace=True)

In [ ]:
# Verify new column names
candidatos_clean.columns.tolist()

<a id='categorical'></a>
### 10. Define categorical columns 

Convert to categorical the `escolaridad, sit_academica` columns

In [ ]:
#Convert to ordered categorical type with custom ordering:
cat_escolaridad = pd.api.types.CategoricalDtype(categories=
                                              ['SIN INSTRUCCIÓN','SABER LEER Y ESCRIBIR','PRIMARIA'
                                              'SECUNDARIA/SEC. TÉCNICA', 'CARRERA COMERCIAL','CARRERA TÉCNICA',
                                               'PROFESIONAL TÉCNICO (CONALEP)', 'PREPA O VOCACIONAL',
                                              'T. SUPERIOR UNIVERSITARIO','LICENCIATURA', 'MAESTRÍA', 'DOCTORADO'],
                                              ordered=True)
candidatos_clean.escolaridad = candidatos_clean.escolaridad.astype(cat_escolaridad);

In [ ]:
candidatos_clean.escolaridad.dtype

For `sit_academica` we'll unify 'NINGUNO' and 'NO ESPECIFICADO'

In [ ]:
candidatos_clean.sit_academica.unique()

In [ ]:
candidatos_clean = candidatos_clean.replace({'sit_academica':{'NINGUNO':'NO ESPECIFICADO'}})

In [ ]:
#Convert to ordered categorical type with custom ordering:
cat_sit_academica = pd.api.types.CategoricalDtype(categories=
                                              ['NO ESPECIFICADO','ESTUDIANTE','DIPLOMA O CERTIFICADO','TRUNCA',
                                              'PASANTE', 'TITULADO'], ordered=True)
candidatos_clean.sit_academica = candidatos_clean.sit_academica.astype(cat_sit_academica);

In [ ]:
candidatos_clean.sit_academica.unique()

<a id='store'></a>[Return to Table of Contents](#table)
## Storing Data

First, we'll get a sample of `candidatos_clean` to verify it is the way we need it.

In [ ]:
candidatos_clean.sample(5)

Then proceed to save it. Make sure to specify index=False and utf-8 encoding.

utf-8 ensures that special characters in Spanish as accents and ñ will be saved.

In [ ]:
candidatos_clean.to_csv('candidatos_clean.csv', index=False, encoding='utf-8')

<a id='estados'></a>
[Return to Table of Contents](#table)
## Analysis and visualization
After wrangling and cleaning data from `candidatos` we can try some univariate exploration.

In [ ]:
# Run this line if you don't have the dataframe postulaciones_clean
postulaciones_clean = pd.read_csv('postulaciones_clean.csv', encoding='utf-8')

### ¿Qué oficinas de SNE tienen más candidatos registrados?

In [ ]:
candidatos_clean.groupby('entidadfed_sne')['id'].count().plot.bar(figsize=(8,5));


### ¿Cuál es la escolaridad de candidatos por sexo ?

In [ ]:
candidatos_clean.groupby(['escolaridad','sexo'])['id'].count().unstack().plot.bar(figsize=(10,5));

### ¿Cuántos candidatos hablan alguna lengua indígena ?

In [ ]:
g3 = pd.DataFrame(candidatos_clean.groupby(['lengua_indigena'])['id'].count(), )

In [ ]:
g3 = g3.drop(index='No');

In [ ]:
g3.unstack().plot.bar(figsize=(12,8));

### ¿Cómo es la distribución de edades por sexo?

In [ ]:
g4 = candidatos_clean[['id','sexo','edad']]

In [ ]:
tick_locs = [10,20,30,40,50,50,70,80,90,100]
tick_names = [10,20,30,40,50,50,70,80,90,100]
sns.violinplot(data = g4, x='sexo', y='edad')
plt.yticks(tick_locs,tick_names);

### ¿Cuáles áreas tienen más candidatos registrados?

In [ ]:
areas_count = candidatos_clean.groupby('area', as_index=False)['id'].count();

In [ ]:
top_10_areas = areas_count.nlargest(10,'id')

In [ ]:
top_10_areas.plot.barh(y='id', x='area', figsize=(10,5)  )

## References
pd.isin()
- https://www.geeksforgeeks.org/python-pandas-dataframe-isin/

Obtener y filtrar datos de un dataframe
- http://pyciencia.blogspot.com/2015/05/obtener-y-filtrar-datos-de-un-dataframe.html
- https://medium.com/@rtjeannier/pandas-101-cont-9d061cb73bfc